<a href="https://colab.research.google.com/github/BhuvaneswariRV/MachineLearning/blob/main/Spark_Basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Used the walmart_stock.csv to perform the below tasks

###Install pyspark

In [1]:
!pip install pyspark

In [2]:
#Import SparkSession
from pyspark.sql import SparkSession

In [3]:
session = SparkSession.builder.appName("spark_basics").getOrCreate()

In [4]:
#Read CSV
df = session.read.csv('walmart_stock.csv',inferSchema=True,header=True)

In [5]:
#What are the column names?
df.columns

['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']

In [6]:
#How schema looks like?
df.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



In [7]:
# Print out the first 5 records
df.head(5)

[Row(Date='2012-01-03', Open=59.970001, High=61.060001, Low=59.869999, Close=60.330002, Volume=12668800, Adj Close=52.619234999999996),
 Row(Date='2012-01-04', Open=60.209998999999996, High=60.349998, Low=59.470001, Close=59.709998999999996, Volume=9593300, Adj Close=52.078475),
 Row(Date='2012-01-05', Open=59.349998, High=59.619999, Low=58.369999, Close=59.419998, Volume=12768200, Adj Close=51.825539),
 Row(Date='2012-01-06', Open=59.419998, High=59.450001, Low=58.869999, Close=59.0, Volume=8069400, Adj Close=51.45922),
 Row(Date='2012-01-09', Open=59.029999, High=59.549999, Low=58.919998, Close=59.18, Volume=6679300, Adj Close=51.616215000000004)]

In [8]:
df.describe()

DataFrame[summary: string, Date: string, Open: string, High: string, Low: string, Close: string, Volume: string, Adj Close: string]

In [9]:
df.select(['Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']).describe().show()

+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|summary|              Open|             High|              Low|            Close|           Volume|        Adj Close|
+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|  count|              1258|             1258|             1258|             1258|             1258|             1258|
|   mean| 72.35785375357709|72.83938807631165| 71.9186009594594|72.38844998012726|8222093.481717011|67.23883848728146|
| stddev|  6.76809024470826|6.768186808159218|6.744075756255496|6.756859163732991|  4519780.8431556|6.722609449996857|
|    min|56.389998999999996|        57.060001|        56.299999|        56.419998|          2094900|        50.363689|
|    max|         90.800003|        90.970001|            89.25|        90.470001|         80898100|84.91421600000001|
+-------+------------------+-----------------+--

## Change decimals in describe to 2 decimal points

In [10]:
df.describe().printSchema()

root
 |-- summary: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Open: string (nullable = true)
 |-- High: string (nullable = true)
 |-- Low: string (nullable = true)
 |-- Close: string (nullable = true)
 |-- Volume: string (nullable = true)
 |-- Adj Close: string (nullable = true)



In [11]:
#Changing data types and loading csv with the predefined schema
from pyspark.sql.types import StructField,FloatType,IntegerType,DateType,StructType

In [12]:
schema = [StructField("Date",DateType(),True),StructField("Open",FloatType(),True),StructField("High",FloatType(),True),
          StructField("Low",FloatType(),True),StructField("Close",FloatType(),True),StructField("Volume",IntegerType(),True),
          StructField("Adj Close",FloatType(),True)] 

In [13]:
struct = StructType(schema)

In [14]:
df = session.read.csv('walmart_stock.csv',schema=struct,inferSchema=True,header=True)

In [15]:
df.printSchema()

root
 |-- Date: date (nullable = true)
 |-- Open: float (nullable = true)
 |-- High: float (nullable = true)
 |-- Low: float (nullable = true)
 |-- Close: float (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: float (nullable = true)



In [16]:
from pyspark.sql.functions import format_number

In [17]:
desc = df.describe()
desc.select(desc['summary'],
              format_number(desc['Open'].cast('double'),2).alias('Open'),
              format_number(desc['High'].cast('double'),2).alias('High'),
              format_number(desc['Low'].cast('double'),2).alias('Low'),
              format_number(desc['Close'].cast('double'),2).alias('Close'),
              desc['Volume'].cast('int').alias('Volume')
             ).show()

+-------+--------+--------+--------+--------+--------+
|summary|    Open|    High|     Low|   Close|  Volume|
+-------+--------+--------+--------+--------+--------+
|  count|1,258.00|1,258.00|1,258.00|1,258.00|    1258|
|   mean|   72.36|   72.84|   71.92|   72.39| 8222093|
| stddev|    6.77|    6.77|    6.74|    6.76| 4519780|
|    min|   56.39|   57.06|   56.30|   56.42| 2094900|
|    max|   90.80|   90.97|   89.25|   90.47|80898100|
+-------+--------+--------+--------+--------+--------+



In [18]:
#  new dataframe with a column called HV Ratio that is the ratio of the High Price versus volume of stock traded for a day.
df1 = df.withColumn("HV Ratio",df['High']/df['Volume'])

In [19]:
df1.select('HV Ratio').show()

+--------------------+
|            HV Ratio|
+--------------------+
|4.819714682786927E-6|
|6.290848662516662E-6|
| 4.66941298944916E-6|
| 7.36733843444859E-6|
|8.915604814435727E-6|
|8.644477449144044E-6|
|9.351828386844425E-6|
| 8.29141562102703E-6|
|7.712212051589609E-6|
|7.071764777688419...|
|1.015495462653464...|
|  6.5763540967921E-6|
| 5.90145296180676E-6|
|8.547679390846264E-6|
|8.420709512685392E-6|
|1.041448335142357...|
|8.316075435382035E-6|
|9.721183804158345E-6|
|8.029435987746889E-6|
|6.307432228123159E-6|
+--------------------+
only showing top 20 rows



In [20]:
# What day had the Peak High in Price?
df.orderBy(df["High"].desc()).head(1)[0][0]

datetime.date(2015, 1, 13)

In [21]:
from pyspark.sql.functions import avg,max,min,count,corr,year,month

In [22]:
# mean of  Close
df.select(avg(df['Close'])).show()

+-----------------+
|       avg(Close)|
+-----------------+
|72.38844997363553|
+-----------------+



In [23]:
# max and min of Volume column
df.select(max(df['Volume']),min(df['Volume'])).show()

+-----------+-----------+
|max(Volume)|min(Volume)|
+-----------+-----------+
|   80898100|    2094900|
+-----------+-----------+



In [24]:
# How many days was the Close lower than 60 dollars
df.filter(df['Close']<60).count()

81

In [25]:
# What percentage of the time was the High greater than 80 dollars ?

(df.filter(df["High"]>80).count()/df.count())*100

9.141494435612083

In [26]:
# What is the Pearson correlation between High and Volume?
df.select(corr("High","Volume")).show()

+-------------------+
| corr(High, Volume)|
+-------------------+
|-0.3384326061799556|
+-------------------+



In [27]:
# What is the max High per year?
year = df.withColumn("Year",year("Date"))
max = year.groupBy('Year').max()
max.select('Year','max(High)').show()

+----+---------+
|Year|max(High)|
+----+---------+
|2015|    90.97|
|2013|    81.37|
|2014|    88.09|
|2012|     77.6|
|2016|    75.19|
+----+---------+



In [28]:
# What is the average Close for each Calendar Month?
month = df.withColumn("Month",month("Date"))
monthavg = month.select("Month","Close").groupBy("Month").mean()
monthavg.select("Month","avg(Close)").orderBy('Month').show()

+-----+-----------------+
|Month|       avg(Close)|
+-----+-----------------+
|    1| 71.4480196131338|
|    2|71.30680438169499|
|    3|71.77794376266337|
|    4|72.97361900692894|
|    5|72.30971685445533|
|    6| 72.4953774506191|
|    7|74.43971944078106|
|    8| 73.0298185521906|
|    9|72.18411782208611|
|   10| 71.5785454489968|
|   11|72.11108927207418|
|   12|72.84792482628012|
+-----+-----------------+

